In [1]:
import pandas as pd
import geoplotlib
import folium

In [7]:
# https://blog.prototypr.io/interactive-maps-with-python-part-1-aa1563dbe5a9
df_bike = pd.read_csv('bike_dat_may.csv')

In [3]:
##line map
geoplotlib.graph(df_bike,
                src_lat='start_station_latitude',
                src_lon='start_station_longitude',
                dest_lat='end_station_latitude',
                dest_lon='end_station_longitude',
                color='hot_r',
                alpha=16,
                linewidth=2)
geoplotlib.show()

In [8]:
# count the freqeuncy of the station appeard in pairs.
df_new = pd.pivot_table(df_bike, index = ['start_station_name','end_station_name'], 
                        values = ['start_station_id'], aggfunc = [len])
df_new = df_new.reset_index()
df_new.columns = ['start_station_name','end_station_name', 'Count']
df_new = df_new.sort_values('Count', ascending = [0])
df_new = df_new.reset_index()

In [9]:
location = df_bike.loc[:, ['start_station_name', 'start_station_latitude', 'start_station_longitude']]
location = location.drop_duplicates().reset_index(drop = True)

In [10]:
route = pd.merge(df_new, location, 
                   on = 'start_station_name', how='left')
route = pd.merge(route, location, left_on = 'end_station_name', 
                 right_on = 'start_station_name', how='left')

route = route.loc[:,["start_station_name_x", "start_station_latitude_x", 
                      'start_station_longitude_x', 
                      'start_station_name_y',
                      'start_station_latitude_y',
                      'start_station_longitude_y',
                      'Count']]

In [11]:
route.head()

,start_station_name_x,start_station_latitude_x,start_station_longitude_x,start_station_name_y,start_station_latitude_y,start_station_longitude_y,Count
0,Portobello,55.957004,-3.116888,Portobello,55.957004,-3.116888,372
1,Newkirkgate,55.970704,-3.171624,Newkirkgate,55.970704,-3.171624,109
2,Cramond Foreshore,55.980024,-3.300622,Cramond Foreshore,55.980024,-3.300622,87
3,Portobello,55.957004,-3.116888,Newkirkgate,55.970704,-3.171624,78
4,Newkirkgate,55.970704,-3.171624,Portobello,55.957004,-3.116888,73


In [12]:
len(route)

3092

In [13]:
end = route.iloc[:,[-3,-2]].values.tolist()
start = route.iloc[:,[1,2]].values.tolist()

In [14]:
my_map = folium.Map(location=[55.953251,-3.188267],
                            zoom_start=12,
                            width= '80%')
#add markers
for i in range(10):
    folium.Marker(start[i],popup="Station:{0}".format(route.iloc[:,0].values.tolist()[i])).add_to(my_map)
    folium.PolyLine([start[i], end[i]], weight = route.iloc[:,-1].values.tolist()[i]/20, color = 'red').add_to(my_map)
my_map

In [15]:
my_map = folium.Map(location=[55.953251,-3.188267],
                            zoom_start=12,
                            width= '80%')
#Line Map of the 
for i in range(100):
    folium.PolyLine([start[i], end[i]], weight = route.iloc[:,-1].values.tolist()[i]/30, color = 'red').add_to(my_map)
my_map